<a href="https://colab.research.google.com/github/Ukbang/Build_with_AI_250405/blob/main/ReportAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain langgraph langchain-google-genai langchain-core langchain-community langchain-experimental fpdf pdfplumber

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 991.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.0/138.0 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.2/209.2 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 26.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [2]:
from langchain_core.runnables import RunnableConfig
from langchain.schema import AIMessage
from typing import Annotated, Literal
from typing_extensions import TypedDict
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.prompts import PromptTemplate
from langchain_core.tools import tool
from langchain_community.agent_toolkits import FileManagementToolkit
from langchain_experimental.tools.python.tool import PythonAstREPLTool
from pydantic import BaseModel, Field
from fpdf import FPDF
import random
import pdfplumber
import os
import requests
import warnings

warnings.filterwarnings("ignore")

In [3]:
google_api_key = "" # @param {"type":"string"}
tavily_api_key = "" # @param {"type":"string"}

In [4]:
os.environ["google_api_key"] = google_api_key
os.environ["GOOGLE_API_KEY"] = google_api_key
os.environ["TAVILY_API_KEY"] = tavily_api_key

In [5]:
class State(TypedDict):
    query : Annotated[str, "User Question"]
    answer : Annotated[str, "LLM response"]
    messages : Annotated[list, add_messages]
    tool_call : Annotated[dict, "Tool Call Result"]

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash",
                             temperature=0,
                             convert_system_message_to_human=True)

In [7]:
@tool
def read_pdf(file_path: str) -> str:
    """
    PDF 파일에서 텍스트를 추출하는 도구입니다.
    표 형식 또는 일반 텍스트가 포함된 PDF를 읽고 문자열로 반환합니다.

    file_path 예시: './report.pdf'
    """
    try:
        text = ""
        with pdfplumber.open(file_path) as pdf:
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
        return text.strip() if text.strip() else "❌ PDF에서 텍스트를 추출할 수 없습니다."
    except Exception as e:
        return f"❌ PDF 읽기 오류: {str(e)}"

In [8]:
@tool
def write_pdf(content: str, filename: str = "output.pdf"):
    """
    텍스트를 PDF 파일로 저장하는 도구입니다.
    PDF형태의 문서로 만들어야할 때 이 도구를 사용하세요.
    """

    prompt = PromptTemplate.from_template("""
            당신은 보고서를 작성하는 어시스턴트입니다. 당신에겐 문서 모음이 제공되고 이를 잘 분석하여 보고서를 작성하여야 합니다.
            아래의 content는 문서 모음입니다. 문서의 제목, 본문을 잘 판단하고 정리하여 요약합니다.
            항상 구조화된 출력을 제공하세요.
            항상 마지막엔 인사이트도 첨부합니다.

            content : {content}
            """)

    chain = prompt | llm

    content = chain.invoke({"content":content}).content

    font_url = "https://github.com/google/fonts/raw/main/ofl/notosanskr/NotoSansKR%5Bwght%5D.ttf"
    font_path = "./fonts/NotoSansKR.ttf"

    try:
        os.mkdir("./fonts/")
        response = requests.get(font_url)
        with open(font_path, "wb") as f:
            f.write(response.content)
    except:
        pass

    pdf = FPDF()
    pdf.add_page()
    pdf.set_auto_page_break(auto=True, margin=15)

    font_path = "./fonts/NotoSansKR.ttf"  # <-- 여기에 실제 폰트 파일이 있어야 함

    try:
        pdf.add_font("NotoSans", "", font_path, uni=True)
        pdf.set_font("NotoSans", size=12)
    except:
        raise ValueError("한글 폰트를 등록할 수 없습니다.")

    for line in content.split("\n"):
        pdf.multi_cell(0, 10, line)
    pdf.output(f"./{filename}")

    return f"{filename} 저장 완료"

In [9]:
tools = [TavilySearchResults(max_results=10), PythonAstREPLTool(), write_pdf, read_pdf, *FileManagementToolkit(
                                                                            selected_tools=["file_delete","list_directory"]).get_tools()]
search_tool, code_tool, write_tool, read_tool, delete_tool, listdir_tool= tools

In [120]:
# tools = [TavilySearchResults(include_domains=["naver.com", "google.com"]), PythonAstREPLTool(), write_pdf, read_pdf, *FileManagementToolkit(
#                                                                             selected_tools=["file_delete","list_directory"]).get_tools()]
# search_tool, code_tool, write_tool, read_tool, delete_tool, listdir_tool= tools

In [121]:
# write_tool({"content":"안녕하세요?", "filename":"abctest.pdf"})

In [122]:
# read_tool("abctest.pdf")

In [123]:
# delete_tool("abctest.pdf")

In [10]:
llm_with_tools = llm.bind_tools(tools)

In [11]:
class HistoryChecker(BaseModel):
    """
    이전의 대화 기록을 참고하여 질문에 대해 답변할 수 있는지 판단합니다.
    답변할 수 있다면 "yes", 답변할 수 없다면 "no"를 반환합니다.
    """

    yes_no : Literal["yes", "no"] = Field(..., description="""Use your previous conversation history to determine if you can answer your questions.
    Return "yes" if you can answer, "no" if you can't answer.""")

In [12]:
def shorterm_memory(state:State):

    if len(state["messages"]) > 8:
        history = state["messages"][-8:-1]
    elif len(state["messages"]) == 1:
        history = ""
    else:
        history = state["messages"][:-1]

    return history

In [13]:
def history_check(state:State):

    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변할 수 있는지 판단합니다.
                답변할 수 있다면 "yes", 답변할 수 없다면 "no"를 반환합니다.

                대화 기록 : {history}

                질문 : {query}

                """)

    chain = prompt | history_checker

    history = shorterm_memory(state)

    result = chain.invoke({"history":history,
                            "query":state["query"]})

    return result.yes_no

In [14]:
def memory_chat(state:State):

    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변하세요.
                아래 대화 기록을 첨부합니다.
                대화 기록을 통해 답변이 어렵다면 내부 지식을 참조하세요.

                대화 기록 : {history}

                질문 : {query}

                """)


    chain = prompt | llm

    history = shorterm_memory(state)

    answer = chain.invoke({"history":history,
                           "query":state["query"]})

    if len(state["tool_call"]) == 0:
        return {"answer":answer.content,
                "messages":[answer],
                "tool_call":"사용된 기록 없음."}
    else:
        return {"answer":answer.content,
                "messages":[answer]}

In [15]:
history_checker = llm.with_structured_output(HistoryChecker)

In [16]:
def history_node(state:State):
    if len(state["messages"]) == 1:
        return {"answer":"답변 없음",
                "tool_call":"사용된 도구 없음"}
    else:
        return state

In [17]:
def select(
    state: State,
):

    prompt = PromptTemplate.from_template("""

                이전의 대화 기록을 참고하여 질문에 대해 답변하세요.
                아래 대화 기록을 첨부합니다.
                이전의 대화가 다음에 어떤 도구를 사용해야하는지 힌트가 될 수 있습니다. 꼭 참조하세요.
                도구의 변화가 큰 결과를 가져올 수 있습니다.
                들어온 메시지, 정답, 이전 기록을 모두 분석하여 가장 적절한 도구를 선택하세요.

                대화 기록 : {history}

                최근 사용한 도구 : {tool_name}

                정답 : {answer}

                질문 : {query}

                """)

    chain = prompt | llm_with_tools

    history = shorterm_memory(state)

    result = chain.invoke({"history" : history,
                           "tool_name" : state["tool_call"],
                            "answer": state["answer"],
                            "query": state["query"]})

    if hasattr(result, "tool_calls") and len(result.tool_calls) > 0:
        tool_calls = result.tool_calls

        return {"messages": result,
                "tool_call":tool_calls}
                # "tool_call":AIMessage(content=tool_calls)}
    else:
        return {"messages":AIMessage(content=f"""도구를 선택하지 못했습니다. 적절한 도구를 재선택하세요.
                                        """),
                                    "tool_call":"선택된 도구 없음"}



In [18]:
tool_node = ToolNode(tools)

In [19]:
class AnswerChecker(BaseModel):
    """
    정답 분류기입니다.

    정답이 질문을 해결했는지 여부를 판단합니다.
    질문을 해결하지 못했을 시 해결될 때까지 도구를 이용합니다.

    질문을 해결했다면 "end", 해결하지 못했다면 "tool"을 반환합니다.
    """


    end : Literal["end", "tool"] = Field(..., description="""You are the answer sorter.

                                                                Determine if the correct answer has solved the question.
                                                                If the question is not resolved, use the tool until it is resolved.

                                                                Return "end" if you solved the question, or "tool" if you didn't.""")

In [20]:
answer_checker = llm.with_structured_output(AnswerChecker)

In [21]:
def response(state:State):

    return {"answer":state["messages"][-1]}

In [22]:
def answer_check(state:State):

    prompt = PromptTemplate.from_template("""
    당신은 정답 분류기 어시스턴트입니다.

    정답이 질문을 해결하였는지 여부를 판단합니다.
    질문을 해결하지 못했다면 도구를 이용합니다.

    질문을 해결하였다면 "end", 아니라면 "tool"을 반환합니다.

    기존 History도 참고하여 답변하세요.

    History : {history}

    정답 : {answer}

    질문 : {query}
    """)

    chain = prompt | answer_checker

    history = shorterm_memory(state)

    result = chain.invoke({"history" : history,
                            "answer": state["answer"],
                            "query": state["query"]})

    return result.end

In [23]:
graph_builder = StateGraph(State)

In [24]:
graph_builder.add_node("history_node", history_node)
graph_builder.add_node("memory_chat", memory_chat)
graph_builder.add_node("select", select)
graph_builder.add_node("tools", tool_node)
graph_builder.add_node("response", response)


graph_builder.add_edge(START, "history_node")
graph_builder.add_conditional_edges("history_node",
                            history_check,
                            {"yes":"memory_chat",
                             "no":"select"})
graph_builder.add_edge("select", "tools")
graph_builder.add_edge("tools", "response")
graph_builder.add_edge("memory_chat", "response")
graph_builder.add_conditional_edges("response",
                                    answer_check,
                                    {"end":END,
                                    "tool":"select"});

In [25]:
memory = MemorySaver()

graph = graph_builder.compile(checkpointer=memory)

In [26]:
graph

ReadTimeout: HTTPSConnectionPool(host='mermaid.ink', port=443): Read timed out. (read timeout=10)

In [27]:
def reset_config(limit=20):

    thread_id=random.randint(1,999999)

    config = RunnableConfig(recursion_limit=limit, configurable={"thread_id": thread_id})

    return config

In [28]:
def streaming(query, config, mode="values"):

    result = graph.stream({"messages":("user", query),
                         "query":query}, config=config, stream_mode=mode)

    if mode == "values":
        for step in result:
            for k, v in step.items():
                if k == "messages":
                    v[-1].pretty_print()
    elif mode == "updates":
        for step in result:
            for k,v in step.items():
                print(f"\n\n=== {k} ===\n\n")
                print(v)

    return

In [29]:
config = reset_config()

query = "1+1은 뭔가요?"

streaming(query, config)

================================ Human Message =================================

1+1은 뭔가요?
================================ Human Message =================================

1+1은 뭔가요?
================================== Ai Message ==================================

2
================================== Ai Message ==================================

2


In [30]:
config = reset_config()

query = "피엔티라는 회사에 대해 조사해주세요. 잘 정리된 보고서를 제공해주십시오. pdf파일로 받기를 희망합니다."

streaming(query, config)

================================ Human Message =================================

피엔티라는 회사에 대해 조사해주세요. 잘 정리된 보고서를 제공해주십시오. pdf파일로 받기를 희망합니다.
================================ Human Message =================================

피엔티라는 회사에 대해 조사해주세요. 잘 정리된 보고서를 제공해주십시오. pdf파일로 받기를 희망합니다.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (e9252c04-08ff-4a47-95e0-a68b58c349b2)
 Call ID: e9252c04-08ff-4a47-95e0-a68b58c349b2
  Args:
    query: 피엔티 회사
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "피엔티 - 위키백과, 우리 모두의 백과사전", "url": "https://ko.wikipedia.org/wiki/%ED%94%BC%EC%97%94%ED%8B%B0", "content": "피엔티 주식회사(PNT Co. Ltd.)는 Roll to Roll 기술을 바탕으로 각종 디스플레이 소재용 필름, 2차 전지의 음극 및 분리막 소재, Copper 등의 코팅 및 슬리터 장비를 제작하는 코스닥 상장 기업이다. 매출구성은 소재 81%, 2차전지 18%, 반도체 1% 가량으로 이루어진다. [...] 형태 | 주식회사\n창립 | 2003/12/30 (상장일: 2012/07/06)\n시장 정보 | 한국:137400\n산업 분야 | 디스플레이

In [31]:
config = reset_config()

query = "피엔티의 경쟁사에 대한 정보도 정리된 보고서를 작성해주세요."

streaming(query, config)

================================ Human Message =================================

피엔티의 경쟁사에 대한 정보도 정리된 보고서를 작성해주세요.
================================ Human Message =================================

피엔티의 경쟁사에 대한 정보도 정리된 보고서를 작성해주세요.
================================== Ai Message ==================================

피엔티의 경쟁사에 대한 정보가 필요하므로, 먼저 검색을 통해 관련 정보를 수집해야 합니다. 그 후, 수집된 정보를 바탕으로 보고서를 작성하는 것이 적절합니다. 따라서 먼저 `tavily_search_results_json` 도구를 사용하여 피엔티 경쟁사에 대한 정보를 검색하겠습니다.
Tool Calls:
  tavily_search_results_json (6673cb11-3ab9-4a68-996e-d4e79c184d1d)
 Call ID: 6673cb11-3ab9-4a68-996e-d4e79c184d1d
  Args:
    query: 피엔티 경쟁사
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "피엔티의 사업분석 및 고객사에 대하여 : 네이버 블로그", "url": "https://blog.naver.com/humanindicator/222353810766", "content": "피엔티의 경쟁사는 롤투롤 장비회사의 양대 산맥인 '씨아이에스'다. 주 거래처는 LG이며 최근 노스볼트의 전극공정까지 수주를 따낸 핫한 기업이다. 둘다", "score": 0.8846886}, {"title": "피엔티 - 오렌지보드", "url": "

}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 9
}
].


================================== Ai Message ==================================

도구를 선택하지 못했습니다. 적절한 도구를 재선택하세요.
================================== Ai Message ==================================

피엔티 경쟁사 정보를 검색했고, 이제 이 정보를 바탕으로 보고서를 작성해야 합니다. `write_pdf` 도구를 사용하여 보고서를 작성하겠습니다.
Tool Calls:
  write_pdf (e2709c6d-21c1-43aa-8ea7-5b9e237e7865)
 Call ID: e2709c6d-21c1-43aa-8ea7-5b9e237e7865
  Args:
    content: 피엔티 경쟁사 정보 보고서

피엔티의 경쟁사로는 씨아이에스, 히라노, 윤성에프앤씨, 케이엔에스 등이 있습니다. 씨아이에스는 롤투롤 장비회사의 양대 산맥으로 LG를 주요 거래처로 하고 있습니다. 히라노는 파우치형 배터리에 사용 가능한 넓은 폭의 코터를 생산하는 기업입니다. 윤성에프앤씨, 케이엔에스 등은 재무 비교를 통해 경쟁사로 언급되었습니다.
    filename: 피엔티 경쟁사 정보 보고서.pdf
================================= Tool Message =================================
Name: write_pdf

피엔티 경쟁사 정보 보고서.pdf 저장 완료
================================= Tool Message =================================
Name: write_pdf

피엔티 경쟁사 정보 보고서.pdf 저장 완료


In [33]:
config = reset_config()

query = "현재 폴더의 pdf로 이루어진 파일 모두 삭제해줘"

streaming(query, config)

================================ Human Message =================================

현재 폴더의 pdf로 이루어진 파일 모두 삭제해줘
================================ Human Message =================================

현재 폴더의 pdf로 이루어진 파일 모두 삭제해줘
================================== Ai Message ==================================
Tool Calls:
  list_directory (8739ea30-91a8-4ce7-b005-14d9ed52bfa7)
 Call ID: 8739ea30-91a8-4ce7-b005-14d9ed52bfa7
  Args:
================================= Tool Message =================================
Name: list_directory

.config
피엔티 경쟁사 정보 보고서.pdf
fonts
피엔티 보고서.pdf
sample_data
================================= Tool Message =================================
Name: list_directory

.config
피엔티 경쟁사 정보 보고서.pdf
fonts
피엔티 보고서.pdf
sample_data
================================== Ai Message ==================================

현재 폴더에 있는 PDF 파일들을 삭제하기 위해 `file_delete` 도구를 사용해야 합니다. 먼저 삭제할 파일 목록을 확인했으니, 이제 각 파일을 삭제하도록 하겠습니다.
Tool Calls:
  file_delete (169a137c-7fe0-41e2-ae00-7420c7f492fc)
 Call ID: 1

In [35]:
config = reset_config()

code = """
아래 코드 실행시켜주세요.

```python

result = 0

for i in range(20):
    print(f"{i+1}번째 출력: ", i+1)
    result += i

print("최종 결과: ", result)

```
"""

streaming(code, config)

================================ Human Message =================================


아래 코드 실행시켜주세요.

```python

result = 0

for i in range(20):
    print(f"{i+1}번째 출력: ", i+1)
    result += i

print("최종 결과: ", result)

```

================================ Human Message =================================


아래 코드 실행시켜주세요.

```python

result = 0

for i in range(20):
    print(f"{i+1}번째 출력: ", i+1)
    result += i

print("최종 결과: ", result)

```

================================== Ai Message ==================================
Tool Calls:
  python_repl_ast (809348e2-5b69-4afa-b071-47c450d2bf0d)
 Call ID: 809348e2-5b69-4afa-b071-47c450d2bf0d
  Args:
    query: 
result = 0

for i in range(20):
    print(f"{i+1}번째 출력: ", i+1)
    result += i

print("최종 결과: ", result)
1번째 출력:  1
2번째 출력:  2
3번째 출력:  3
4번째 출력:  4
5번째 출력:  5
6번째 출력:  6
7번째 출력:  7
8번째 출력:  8
9번째 출력:  9
10번째 출력:  10
11번째 출력:  11
12번째 출력:  12
13번째 출력:  13
14번째 출력:  14
15번째 출력:  15
16번째 출력:  16
17번째 출력:  17
18번째 출력:  18
19번째 출력:  19
20번째

In [36]:
config = reset_config()

streaming("""
          모두의연구소는 어떤 곳이야?
          깔끔하게 정리해서 레포트로 만들어줘.
          레포트의 형식은 pdf로 저장해주면 돼.
          이름은 "모두의연구소_레포트.pdf"로 해줘.""", config)

================================ Human Message =================================


          모두의연구소는 어떤 곳이야?
          깔끔하게 정리해서 레포트로 만들어줘.
          레포트의 형식은 pdf로 저장해주면 돼.
          이름은 "모두의연구소_레포트.pdf"로 해줘.
================================ Human Message =================================


          모두의연구소는 어떤 곳이야?
          깔끔하게 정리해서 레포트로 만들어줘.
          레포트의 형식은 pdf로 저장해주면 돼.
          이름은 "모두의연구소_레포트.pdf"로 해줘.
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (0ce89b6d-15aa-45ea-9909-e5fe24aea6d3)
 Call ID: 0ce89b6d-15aa-45ea-9909-e5fe24aea6d3
  Args:
    query: 모두의연구소
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "모두의연구소 2025년 기업정보 - 직원수, 근무환경, 복리후생 등", "url": "https://www.jobkorea.co.kr/recruit/co_read/c/modulabs", "content": "기업소개. 모두의연구소는 AI 교육인 아이펠과 작년에 새롭게 런칭된 SW 교육, 그리고 다양한 주제가 있는 커뮤니티를 기반으로 교육 및 연구 서비스를 제공하고 있어요. '", "score"

In [38]:
config = reset_config()

query = "'모두의연구소_레포트.pdf'라는 파일 삭제해줘"

streaming(query, config)

================================ Human Message =================================

'모두의연구소_레포트.pdf'라는 파일 삭제해줘
================================ Human Message =================================

'모두의연구소_레포트.pdf'라는 파일 삭제해줘
================================== Ai Message ==================================
Tool Calls:
  file_delete (1eda2157-d41a-4929-8bd0-cf9ab1e5aaff)
 Call ID: 1eda2157-d41a-4929-8bd0-cf9ab1e5aaff
  Args:
    file_path: 모두의연구소_레포트.pdf
================================= Tool Message =================================
Name: file_delete

File deleted successfully: 모두의연구소_레포트.pdf.
================================= Tool Message =================================
Name: file_delete

File deleted successfully: 모두의연구소_레포트.pdf.
